<a href="https://colab.research.google.com/github/GiulioAppetito/Progetto_MOBD_2122/blob/main/prova9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My\ Drive/ProgettoMOBD

/content/drive/My Drive/ProgettoMOBD


In [47]:
import sklearn
import pandas as pd
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn_pandas import CategoricalImputer
from sklearn.ensemble import IsolationForest, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.svm import SVC, LinearSVC
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn import tree
import graphviz
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

np.random.seed = 123

In [ ]:
# LETTURA DATASET
data = pd.read_csv("train.csv", sep=",")
print(data.head())
# print("\nLabels: {}".format(np.unique(data.Y, return_counts=True)))

# print("\n# of NaNs values for each column:")
# print(data.isnull().sum(axis=0))

# dropping nan values: lowers the performance of circa 10%
# data = data.dropna()

x = data.iloc[:, :-1].to_numpy()
y = data.iloc[:, -1].to_numpy()
# print(x.shape)
# print(y.shape)

# stratify: to mantain the same mean as y for y_tr and y_ts
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.3, random_state=258, stratify=y)

# print(x_tr.shape)
# print(y_tr.shape)
# print(x_ts.shape)
# print(y_ts.shape)

print(np.unique(y, return_counts=True)[1]/len(y))
print(np.unique(y_tr, return_counts=True)[1]/len(y_tr))
print(np.unique(y_ts, return_counts=True)[1]/len(y_ts))

In [57]:
# valori nan
imp = SimpleImputer(missing_values=np.nan, strategy = 'most_frequent')
x_tr_notnan = imp.fit_transform(x_tr)
x_ts_notnan = imp.transform(x_ts)

In [ ]:
#gestione features categoriche
#dato che però questi sono ndarrays, per fare la one hot coded technique dobbiamo usare dataframes

x_tr_df = pd.DataFrame(x_tr_notnan)
x_ts_df = pd.DataFrame(x_ts_notnan)

x_tr_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']
x_ts_df.columns=['F0','F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','F11','F12','F13']

# one hot code

final_df_tr = pd.concat([x_tr_df,pd.get_dummies(x_tr_df['F1'])],axis = 1)
final_df_tr.drop('F1', axis=1, inplace=True)

final_df_tr = pd.concat([final_df_tr,pd.get_dummies(x_tr_df['F7'])],axis = 1)
final_df_tr.drop('F7', axis=1, inplace=True)

final_df_tr = pd.concat([final_df_tr,pd.get_dummies(x_tr_df['F9'])],axis = 1)
final_df_tr.drop('F9', axis=1, inplace=True)

final_df_tr = pd.concat([final_df_tr,pd.get_dummies(x_tr_df['F8'])],axis = 1)
final_df_tr.drop('F8', axis=1, inplace=True)

final_df_tr = pd.concat([final_df_tr,pd.get_dummies(x_tr_df['F5'])],axis = 1)
final_df_tr.drop('F5', axis=1, inplace=True)

final_df_tr = pd.concat([final_df_tr,pd.get_dummies(x_tr_df['F6'])],axis = 1)
final_df_tr.drop('F6', axis=1, inplace=True)

final_df_tr = pd.concat([final_df_tr,pd.get_dummies(x_tr_df['F13'])],axis = 1)
final_df_tr.drop('F13', axis=1, inplace=True)

final_df_tr.drop('F3', axis=1, inplace=True)

#print(final_df_tr.head())


final_df_ts = pd.concat([x_ts_df,pd.get_dummies(x_ts_df['F1'])],axis = 1)
final_df_ts.drop('F1', axis=1, inplace=True)

final_df_ts = pd.concat([final_df_ts,pd.get_dummies(x_ts_df['F7'])],axis = 1)
final_df_ts.drop('F7', axis=1, inplace=True)

final_df_ts = pd.concat([final_df_ts,pd.get_dummies(x_ts_df['F9'])],axis = 1)
final_df_ts.drop('F9', axis=1, inplace=True)

final_df_ts = pd.concat([final_df_ts,pd.get_dummies(x_ts_df['F8'])],axis = 1)
final_df_ts.drop('F8', axis=1, inplace=True)

final_df_ts = pd.concat([final_df_ts,pd.get_dummies(x_ts_df['F5'])],axis = 1)
final_df_ts.drop('F5', axis=1, inplace=True)

final_df_ts = pd.concat([final_df_ts,pd.get_dummies(x_ts_df['F6'])],axis = 1)
final_df_ts.drop('F6', axis=1, inplace=True)

final_df_ts = pd.concat([final_df_ts,pd.get_dummies(x_ts_df['F13'])],axis = 1)
final_df_ts.drop('F13', axis=1, inplace=True)

final_df_ts.drop('F3', axis=1, inplace=True)

#print(final_df_ts.tail())
#print(final_df_tr.tail())

# la align la stiamo usando perchè avendo usato get_dummies purtroppo se qualche valore che compare in uno dei due set
# non è presente nell'altro, non verrà generata la colonna corrispondente, per cui non è detto che i due dataframe che si 
# vanno ad usare abbiamo stesso insieme di colonne.
# tantomeno se si va ad usare un dataset che non si conosce, non si può sapere se in futuro ci saranno valori di cui non 
# si è tenuto conto etc
x_tr_df, x_ts_df = final_df_tr.align(final_df_ts, join="outer", axis=1,fill_value=0)
print(x_tr_df.tail())
print(x_ts_df.tail())


In [ ]:
# SCALING
print("Robust Scaler")
scaler = RobustScaler()
x_tr_scaled = scaler.fit_transform(x_tr_df)
x_ts_scaled = scaler.transform(x_ts_df)

print("Media distribuzione di partenza: {}".format(np.mean(x_tr_df, axis=0)))
print("Media distribuzione scalata: {}".format(np.mean(x_tr_scaled, axis=0)))

print("Deviazione standard distribuzione di partenza: {}".format(np.std(x_tr_df, axis=0)))
print("Deviazione standard distribuzione scalata: {}".format(np.std(x_tr_scaled, axis=0)))

print("Media distribuzione di partenza: {}".format(np.mean(x_ts_df, axis=0)))
print("Media distribuzione scalata: {}".format(np.mean(x_ts_scaled, axis=0)))

print("Deviazione standard distribuzione di partenza: {}".format(np.std(x_ts_df, axis=0)))
print("Deviazione standard distribuzione scalata: {}".format(np.std(x_ts_scaled, axis=0)))

In [60]:
# BALANCING: oversampling

#balancer = SMOTE()
#x_tr_balanced, y_tr_balanced = balancer.fit_resample(x_tr_scaled, y_tr)

In [ ]:
# ANOMALY DETECTION CON ISOLATION FOREST
print("Initial distribution")
print(len(x_tr))
print(np.unique(y_tr, return_counts=True)[1]/len(y_tr))

anomaly_detector = IsolationForest()
anomaly_detector.fit(x_tr_scaled)
is_inlier = anomaly_detector.predict(x_tr_scaled)

print(np.unique(is_inlier, return_counts=True))
x_tr_not_anomalous = x_tr_scaled[is_inlier==1,:]
y_tr_not_anomalous = y_tr[is_inlier==1]

print("Distribution after anomaly detection")
print(len(x_tr_not_anomalous))
print(np.unique(y_tr_not_anomalous, return_counts=True)[1]/len(y_tr_not_anomalous))

In [ ]:
# BALANCING: undersampling
n_classes = len(np.unique(y_tr))
fig, ax = plt.subplots()
sizes = [len(y_tr[y_tr==i]) for i in range(n_classes)]
class_names = [chr(ord('A')+i) for i in range(n_classes)]

ax.pie(sizes, labels=class_names, autopct='%1.1f%%',shadow=True,startangle=90)
ax.axis("equal")
plt.show()

sampler = ClusterCentroids(random_state=42)
x_tr_balanced, y_tr_balanced = sampler.fit_resample(x_tr_not_anomalous, y_tr_not_anomalous)

print("Size del dataset prima del sampler: {}".format(x_tr_not_anomalous.shape[0]))
print("Size del dataset dopo il sampler: {}".format(x_tr_balanced.shape[0]))

fig, ax = plt.subplots()
sizes = [len(y_tr_balanced[y_tr_balanced==i]) for i in range(n_classes)]
class_names = [chr(ord('A')+i) for i in range(n_classes)]

ax.pie(sizes, labels=class_names, autopct='%1.1f%%',shadow=True,startangle=90)
ax.axis("equal")
plt.show()

In [ ]:

def train_evaluate(model, x_tr, y_tr, x_ts, y_ts):
  model.fit(x_tr, y_tr)
  y_tr_pred = model.predict(x_tr)
  y_ts_pred = model.predict(x_ts)
  print("Accuracy on training set {}".format(accuracy_score(y_tr, y_tr_pred)))
  print("F1 score on training set {}".format(f1_score(y_tr, y_tr_pred)))
  print(confusion_matrix(y_tr, y_tr_pred))
  print("Accuracy on test set {}".format(accuracy_score(y_ts, y_ts_pred)))
  print("F1 score on test set {}".format(f1_score(y_ts, y_ts_pred)))
  print(confusion_matrix(y_ts, y_ts_pred))
print("******* SVC")
classifier = SVC(kernel='rbf', C = 0.4)
print("Training on scaled data")
train_evaluate(classifier, x_tr_scaled, y_tr, x_ts_scaled, y_ts)
print("******* SVC")

classifier = SVC(kernel='rbf', C = 0.4)
print("Training on balanced data")
train_evaluate(classifier, x_tr_balanced, y_tr_balanced, x_ts_scaled, y_ts)
print("******* SVC")

classifier = SVC(kernel='rbf', C = 0.4)
print("Training on not anomalous data")
train_evaluate(classifier, x_tr_not_anomalous, y_tr_not_anomalous, x_ts_scaled, y_ts)


In [63]:
# CLASSIFICATION: DECISION TREES
print("******* DecisionTreeClassifier")
classifier = DecisionTreeClassifier(criterion="gini", max_depth=5)
train_evaluate(classifier, x_tr_not_anomalous, y_tr_not_anomalous, x_ts_scaled, y_ts)

******* DecisionTreeClassifier
Accuracy on training set 0.846704048743874
F1 score on training set 0.6131907308377896
[[16425   764]
 [ 2708  2752]]
Accuracy on test set 0.846043607329307
F1 score on test set 0.6125708397733127
[[7076  341]
 [1163 1189]]


In [65]:
# CLASSIFICATION: METODI ENSEMBLE
print("******* RandomForestClassifier")
classifier = RandomForestClassifier(max_depth=5, n_estimators=100)
train_evaluate(classifier, x_tr_not_anomalous, y_tr_not_anomalous, x_ts_scaled, y_ts)
print("******* GradientBoostingClassifier")
classifier = GradientBoostingClassifier(max_depth=5, n_estimators=100)
train_evaluate(classifier, x_tr_not_anomalous, y_tr_not_anomalous, x_ts_scaled, y_ts)
print("******* AdaBoostClassifier")
classifier = AdaBoostClassifier(n_estimators=100)
train_evaluate(classifier, x_tr_not_anomalous, y_tr_not_anomalous, x_ts_scaled, y_ts)

******* RandomForestClassifier
Accuracy on training set 0.8396838712525939
F1 score on training set 0.5638438438438438
[[16671   518]
 [ 3113  2347]]
Accuracy on test set 0.8414371993039206
F1 score on test set 0.5761969904240766
[[7167  250]
 [1299 1053]]
******* GradientBoostingClassifier
Accuracy on training set 0.8846306680206631
F1 score on training set 0.7384646181563407
[[16347   842]
 [ 1771  3689]]
Accuracy on test set 0.8741938785955574
F1 score on test set 0.7174063002989192
[[6980  437]
 [ 792 1560]]
******* AdaBoostClassifier
Accuracy on training set 0.8646739370391628
F1 score on training set 0.686893451833691
[[16222   967]
 [ 2098  3362]]
Accuracy on test set 0.8662094380182209
F1 score on test set 0.6926875146955092
[[6989  428]
 [ 879 1473]]
